In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Integer, String, Float
import psycopg2

file_paths = {
    "2019": '2019.xlsx',
    "2020": '2020.xlsx',
    "2021": '2021.xlsx',
    "2022": '2022.xlsx'
}

# Loading data for each year
data_2019 = pd.read_excel(file_paths["2019"])
data_2020 = pd.read_excel(file_paths["2020"])
data_2021 = pd.read_excel(file_paths["2021"])
data_2022 = pd.read_excel(file_paths["2022"])

# Add 'Year' column to each DataFrame
data_2019['Year'] = 2019
data_2020['Year'] = 2020
data_2021['Year'] = 2021
data_2022['Year'] = 2022

# Concatenate all DataFrames
all_data = pd.concat([data_2019, data_2020, data_2021, data_2022])

all_data.head(10)

In [ ]:
# Concatenate all DataFrames
all_data = pd.concat([data_2019, data_2020, data_2021, data_2022])

all_data

In [ ]:
# Copy Dataframe
cleaned_data = all_data.copy()

# Assuming 'all_data' is your DataFrame and 'item' is the column you're checking for NaN values
cleaned_data = cleaned_data.dropna(subset=['Item'])
cleaned_data.head(10)

In [ ]:
# Copy Dataframe
all_data_columns_renamed = cleaned_data.copy()

# Rename Columns
all_data_columns_renamed = all_data_columns_renamed.rename(columns={
    'All\nconsumer\nunits': 'all_consumer_units',
    'Birth year\nof 1997\nor later': 'gen_z',
    'Birth year\nfrom 1981\nto 1996': 'millennials',
    'Birth year\nfrom 1965\nto 1980': 'gen_x',
    'Birth year\nfrom 1946\nto 1964': 'baby_boomers',
    'Birth year\nof 1945\nor earlier': 'silent_generation',
    'Year': 'year',
    'Item': 'item'
    })

all_data_columns_renamed.head(10)

In [ ]:
# Copy Dataframe
df_year_reordered = all_data_columns_renamed.copy()

# Ensure 'Year' is the first column
column_order = ['year'] + [col for col in df_year_reordered.columns if col != 'year']
column_order = df_year_reordered[column_order]

# Display the reordered and reindexed DataFrame
column_order.head(10)


In [ ]:
# Copy Dataframe
null_check = column_order.copy()

# Define the subset of columns to check for null values (all columns except 'Year')
subset_columns = [col for col in null_check.columns if col != 'year']

# Drop rows where all specified columns are null
row_drop = null_check.dropna(how='all', subset=subset_columns)
# Resetting the index
row_drop.reset_index(drop=True, inplace=True)

# Display the cleaned data
row_drop.head(10)


In [ ]:
## Append Main Category and Subcategory 

# Copy Dataframe
column_init =row_drop.copy()

# Initialize columns for 'Main Category' and 'Subcategory' in 'all_data_drop_blanks'
column_init['main_category'] = np.nan
column_init['subcategory'] = np.nan

main_category = None
current_subcategory = None
for i in range(len(column_init)):
    if pd.isnull(column_init.loc[i, 'all_consumer_units']):
        if main_category is None or (i+1 < len(column_init) and pd.isnull(column_init.loc[i+1, 'all_consumer_units'])):
            # Current row is identified as a main category
            main_category = column_init.loc[i, 'item']
            current_subcategory = None  # Reset subcategory for a new main category
        else:
            # Current row is identified as a subcategory
            current_subcategory = column_init.loc[i, 'item']
    column_init.loc[i, 'main_category'] = main_category
    if current_subcategory is not None:
        column_init.loc[i, 'subcategory'] = current_subcategory
    else:
        column_init.loc[i, 'subcategory'] = column_init.loc[i, 'item']  # Use item as subcategory if no subcategory defined

# Display the DataFrame
column_init.head(10)

In [ ]:
# Copy Dataframe
item_df = column_init.copy()

# For rows not considered subcategories, fill 'Subcategory' with 'Item'
item_df['subcategory'].fillna(item_df['item'], inplace=True)

# Display the DataFrame
item_df.head(10)

In [ ]:
# Copy Dataframe
nans_clean = item_df.copy()

# Optionally, to clean up, fill remaining NaNs in 'Main Category'
nans_clean['main_category'].fillna(method='ffill', inplace=True)

# Display the DataFrame
nans_clean.head(10)

In [ ]:
# Copy Dataframe
colon_dropped = nans_clean.copy()


# Define a function to remove ':' from non-empty strings
def remove_colons(cell_value):
    if isinstance(cell_value, str):
        return cell_value.replace(':', '')
    else:
        return cell_value

# Apply the function to the entire DataFrame
all_data_drop_colons  = colon_dropped.applymap(remove_colons)

# Remove rows containing 'b/', '/c', or 'a/' in any column
filtered_df = all_data_drop_colons[~all_data_drop_colons.apply(lambda row: row.astype(str).str.contains('b/|/c|a/').any(), axis=1)]

# Display the filtered DataFrame
filtered_df.head(10)

In [ ]:
# Copy Dataframe
remaining_nans = filtered_df.copy()

# Assuming 'all_data' is your DataFrame and 'item' is the column you're checking for NaN values
final_data = remaining_nans.dropna(subset=['all_consumer_units'])

# Display the cleaned data
final_data.head(10)

In [ ]:
# Copy Dataframe
index_reset = final_data.copy()

#Index Reset
index_reset.reset_index()

# rename data frame
final_df = index_reset

# Display the cleaned data
final_data.head(10)



In [ ]:
# Database connection parameters
host = "192.168.50.231"  # or your host, e.g., "127.0.0.1"
#dbname = "consumer_data"
user = "postgres"
password = "postgres"
port = "5432"  # default PostgreSQL port

In [ ]:
# Connect to your database
conn = psycopg2.connect( user=user, password=password, host=host, port=port)


In [ ]:
# Cursor to execute commands
cur = conn.cursor()

In [ ]:
# Check if the table exists and create it if it does not
cur.execute("""
    SELECT EXISTS (
        SELECT FROM pg_tables
        WHERE schemaname = 'public' AND tablename  = 'consumer_data'
    );
""")

exists = cur.fetchone()[0]

if not exists:
    cur.execute("""
        CREATE TABLE consumer_data (
            id SERIAL PRIMARY KEY,
            Year INTEGER,
            Item TEXT,
            All_Consumer_Units FLOAT,
            Gen_Z FLOAT,
            Millennials FLOAT,
            Gen_X FLOAT,
            Baby_Boomers FLOAT,
            Silent_Generation FLOAT,
            Main_Category TEXT,
            Subcategory TEXT
        );
    """)
    print("Table 'consumer_data' created.")
else:
    print("Table 'consumer_data' already exists.")

In [ ]:
# Commit changes
conn.commit()

In [ ]:
# Close cursor and connection
cur.close()
conn.close()

In [ ]:
# SQLAlchemy connection string
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

In [ ]:
# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

In [ ]:
# Convert DataFrame column names to lowercase to ensure consistency
#df.columns = [col.lower() for col in df.columns]

In [ ]:
# Explicitly define the dtype mapping; adjust types as necessary
dtype_mapping = {
    'id': Integer(),
    'year': Integer(),
    'item': String(),
    'all_consumer_units': Float(),
    'gen_z': Float(),
    'millennials': Float(),
    'gen_x': Float(),
    'baby_boomers': Float(),
    'silent_generation': Float(),
    'maincategory': String(),
    'subcategory': String()
}

In [ ]:
# copy dataframe
double_precision = final_data.copy()

# Define a function to handle invalid values in a column
def clean_double_precision_value(value, column_name):
    if isinstance(value, str) and (value == 'd/' or value == 'c/'):
        # Handle 'd/' and 'c/' by returning None
        return None
    try:
        # Try to convert the value to a float (double precision)
        return float(value)
    except (ValueError, TypeError):
        # Handle other invalid values here, if needed
        print(f"Invalid value in column '{column_name}': {value}")
        return None

# Columns with double precision values (customize this based on your DataFrame)
double_precision_columns = ['gen_z', 'millennials', 'gen_x', 'baby_boomers', 'silent_generation']

# Apply the cleaning function to the specified columns
for column in double_precision_columns:
    double_precision[column] = double_precision.apply(lambda row: clean_double_precision_value(row[column], column), axis=1)


# Use the `to_sql` method to insert data, applying the dtype mapping
double_precision.to_sql('consumer_data', engine, if_exists='replace', index=False, dtype=dtype_mapping)

print("Data successfully inserted into the database.")

# Display the cleaned DataFrame
double_precision.head(10)
